In [1]:
x=1

In [2]:
# run in env monae_simba

import os
import sys
sys.path.append("../../../monae")

print(sys.path)

import src.emb_init as si
import anndata
import matplotlib.pyplot as plt
import time 
import muon as mu
import scanpy as sc
import numpy as np
import networkx as nx
import warnings
import torch
import scglue
import itertools

['/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/monae_simba/lib/python310.zip', '/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/monae_simba/lib/python3.10', '/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/monae_simba/lib/python3.10/lib-dynload', '', '/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/monae_simba/lib/python3.10/site-packages', '../../../monae']


/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/monae_simba/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/monae_simba/lib/python3.10/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/monae_simba/lib/python3.10/site-packages/muon/_core/preproc.py:31: FutureWarning: `__version__` is deprecated, use `importlib.metadata.version('scanpy')` instead
  if Version(scanpy.__version__) < Version("1.10"):


## actual data

In [4]:
gene_adata_names = ['adata_gene_25', 'adata_gene_50', 'adata_gene_75']

for file_name in gene_adata_names:
    split = file_name.split('_')[-1]

    prot_name = file_name.replace('gene', 'protein')

    adata_CPro = sc.read(f'/lustre/groups/ml01/code/anastasia.litinetskaya/multigrate_reproducibility/imputation/pipeline/data/input/cite/{prot_name}.h5ad')
    adata_CG = sc.read(f'/lustre/groups/ml01/code/anastasia.litinetskaya/multigrate_reproducibility/imputation/pipeline/data/input/cite/{file_name}.h5ad')

    adata_CPro = adata_CPro[adata_CPro.obs['split']=='train'].copy()
    adata_CG = adata_CG[adata_CG.obs['split']=='train'].copy()

    adata_CG.var_names = adata_CG.var_names + '_gex'
    adata_CPro.var_names = adata_CPro.var_names + '_adt'

    adata_CG.X = adata_CG.layers['counts'].copy()
    adata_CPro.X = adata_CPro.layers['counts'].copy()

    # monae normalization
    si.pp.normalize(adata_CPro,method='lib_size')
    si.pp.log_transform(adata_CPro)
    si.tl.discretize(adata_CPro,n_bins=5)

    si.pp.normalize(adata_CG,method='lib_size')
    si.pp.log_transform(adata_CG)
    si.tl.discretize(adata_CG,n_bins=5)

    workdir = f"./gnn_result_{split}"
    si.settings.set_workdir(workdir)

    start_time = time.time()
    si.tl.gen_graph(list_CPro=[adata_CPro],
                    list_CG=[adata_CG],
                    copy=False,
                    use_highly_variable=False,
                    use_top_pcs=False,
                    dirname='graph0')

    si.tl.pbg_train(
        auto_wd=True, 
        save_wd=True, 
        output='model', 
        pre_embeddings=None,
    )
    end_time = time.time()

    time_taken = end_time - start_time
    print('Time taken:', time_taken)

    dict_adata = si.read_embedding()

    adata_C = dict_adata['C']  # embeddings for cells
    adata_G = dict_adata['G']  # embeddings for genes
    adata_Pro = dict_adata['Pro']  # embeddings for proteins

    adata_all = si.tl.embed(adata_ref=adata_C, list_adata_query=[adata_G, adata_Pro])

    adata_G.X = adata_all[adata_G.obs_names,:].X
    adata_Pro.X = adata_all[adata_Pro.obs_names,:].X

    adata_CrnaCadt = si.tl.infer_guidance_edges(adata_G, adata_Pro, n_components=15, k=50)

    graph = nx.MultiDiGraph()
    for i in range(adata_CrnaCadt.layers['conn'].shape[0]):
        for j in range(adata_CrnaCadt.layers['conn'].shape[1]):
            if adata_CrnaCadt.layers['conn'][i, j] != 0:
                graph.add_edge(adata_CrnaCadt.obs_names[i], adata_CrnaCadt.var_names[j], weight=1.0, sign=1)

    graph = scglue.graph.compose_multigraph(graph, graph.reverse())

    for i in itertools.chain(adata_CG.var_names, adata_CPro.var_names):
        graph.add_edge(i, i, weight=1.0, sign=1)

    print(graph)

    scglue.graph.check_graph(graph, [adata_CG, adata_CPro])

    nx.write_graphml(graph, f"../pipeline/data/input/cite/guidance-hvf_{split}.graphml.gz")

Saving results in: ./gnn_result_25
relation0: source: C, destination: Pro
#edges: 233800
relation1: source: C, destination: Pro
#edges: 466374
relation2: source: C, destination: Pro
#edges: 706888
relation3: source: C, destination: Pro
#edges: 720215
relation4: source: C, destination: Pro
#edges: 406645
relation5: source: C, destination: G
#edges: 330403
relation6: source: C, destination: G
#edges: 1261857
relation7: source: C, destination: G
#edges: 1360094
relation8: source: C, destination: G
#edges: 690817
relation9: source: C, destination: G
#edges: 183384
Total number of edges: 6360477
Writing graph file "pbg_graph.txt" to "./gnn_result_25/pbg/graph0" ...
Finished.
Auto-estimated weight decay is 0.005571
`.settings.pbg_params['wd']` has been updated to 0.005571
Converting input data ...
[2025-12-14 11:40:20.064006] Using the 10 relation types given in the config
[2025-12-14 11:40:20.064689] Searching for the entities in the edge files...
[2025-12-14 11:40:26.755241] Entity type C:

/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/monae_simba/lib/python3.10/site-packages/torchbiggraph/tensorlist.py:86: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  if data.numel() == 0 and data.storage().size() == 0:


[2025-12-14 11:41:03.999018] - Processed 6360477 edges in total
Starting training ...


/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/monae_simba/lib/python3.10/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/monae_simba/lib/python3.10/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/monae_simba/lib/python3.10/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If 

Finished
Time taken: 541.7236838340759
Performing softmax transformation for query data 0;
Performing softmax transformation for query data 1;
#shared features: 50
Performing randomized SVD ...
(2000, 50)
(134, 50)
Searching for mutual nearest neighbors ...
6700 edges are selected


...

MultiDiGraph with 2134 nodes and 15534 edges
[INFO] check_graph: Checking variable coverage...
[INFO] check_graph: Checking edge attributes...
[INFO] check_graph: Checking self-loops...
[INFO] check_graph: Checking graph symmetry...
[INFO] check_graph: All checks passed!
Saving results in: ./gnn_result_75
relation0: source: C, destination: Pro
#edges: 681024
relation1: source: C, destination: Pro
#edges: 1386971
relation2: source: C, destination: Pro
#edges: 2129286
relation3: source: C, destination: Pro
#edges: 2179548
relation4: source: C, destination: Pro
#edges: 1227061
relation5: source: C, destination: G
#edges: 979647
relation6: source: C, destination: G
#edges: 3811664
relation7: source: C, destination: G
#edges: 4058886
relation8: source: C, destination: G
#edges: 2076287
relation9: source: C, destination: G
#edges: 550126
Total number of edges: 19080500
Writing graph file "pbg_graph.txt" to "./gnn_result_75/pbg/graph0" ...
Finished.
Auto-estimated weight decay is 0.001857
`.s

/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/monae_simba/lib/python3.10/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/monae_simba/lib/python3.10/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/monae_simba/lib/python3.10/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If 

Finished
Time taken: 2459.7145307064056
Performing softmax transformation for query data 0;
Performing softmax transformation for query data 1;
#shared features: 50
Performing randomized SVD ...
(2000, 50)
(134, 50)
Searching for mutual nearest neighbors ...
6700 edges are selected


/ictstr01/groups/ml01/code/anastasia.litinetskaya/multigrate_reproducibility/imputation/monae/../../../monae/src/emb_init/tools/_integration.py:135: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata_ref_query = ad.AnnData(X=sim_ref_query,


MultiDiGraph with 2134 nodes and 15534 edges
[INFO] check_graph: Checking variable coverage...
[INFO] check_graph: Checking edge attributes...
[INFO] check_graph: Checking self-loops...
[INFO] check_graph: Checking graph symmetry...
[INFO] check_graph: All checks passed!


In [3]:
# full dataset
gene_adata_names = ['adata_gene_25']

for file_name in gene_adata_names:
    prot_name = file_name.replace('gene', 'protein')

    adata_CPro = sc.read(f'/lustre/groups/ml01/code/anastasia.litinetskaya/multigrate_reproducibility/imputation/pipeline/data/input/cite/{prot_name}.h5ad')
    adata_CG = sc.read(f'/lustre/groups/ml01/code/anastasia.litinetskaya/multigrate_reproducibility/imputation/pipeline/data/input/cite/{file_name}.h5ad')

    adata_CG.var_names = adata_CG.var_names + '_gex'
    adata_CPro.var_names = adata_CPro.var_names + '_adt'

    adata_CG.X = adata_CG.layers['counts'].copy()
    adata_CPro.X = adata_CPro.layers['counts'].copy()

    # monae normalization
    si.pp.normalize(adata_CPro,method='lib_size')
    si.pp.log_transform(adata_CPro)
    si.tl.discretize(adata_CPro,n_bins=5)

    si.pp.normalize(adata_CG,method='lib_size')
    si.pp.log_transform(adata_CG)
    si.tl.discretize(adata_CG,n_bins=5)

    workdir = f"./gnn_result_100"
    si.settings.set_workdir(workdir)

    start_time = time.time()
    si.tl.gen_graph(list_CPro=[adata_CPro],
                    list_CG=[adata_CG],
                    copy=False,
                    use_highly_variable=False,
                    use_top_pcs=False,
                    dirname='graph0')

    si.tl.pbg_train(
        auto_wd=True, 
        save_wd=True, 
        output='model', 
        pre_embeddings=None,
    )
    end_time = time.time()

    time_taken = end_time - start_time
    print('Time taken:', time_taken)

    dict_adata = si.read_embedding()

    adata_C = dict_adata['C']  # embeddings for cells
    adata_G = dict_adata['G']  # embeddings for genes
    adata_Pro = dict_adata['Pro']  # embeddings for proteins

    adata_all = si.tl.embed(adata_ref=adata_C, list_adata_query=[adata_G, adata_Pro])

    adata_G.X = adata_all[adata_G.obs_names,:].X
    adata_Pro.X = adata_all[adata_Pro.obs_names,:].X

    adata_CrnaCadt = si.tl.infer_guidance_edges(adata_G, adata_Pro, n_components=15, k=50)

    graph = nx.MultiDiGraph()
    for i in range(adata_CrnaCadt.layers['conn'].shape[0]):
        for j in range(adata_CrnaCadt.layers['conn'].shape[1]):
            if adata_CrnaCadt.layers['conn'][i, j] != 0:
                graph.add_edge(adata_CrnaCadt.obs_names[i], adata_CrnaCadt.var_names[j], weight=1.0, sign=1)

    graph = scglue.graph.compose_multigraph(graph, graph.reverse())

    for i in itertools.chain(adata_CG.var_names, adata_CPro.var_names):
        graph.add_edge(i, i, weight=1.0, sign=1)

    print(graph)

    scglue.graph.check_graph(graph, [adata_CG, adata_CPro])

    nx.write_graphml(graph, f"../pipeline/data/input/cite/guidance-hvf_100.graphml.gz")

Saving results in: ./gnn_result_100
relation0: source: C, destination: Pro
#edges: 912484
relation1: source: C, destination: Pro
#edges: 1850973
relation2: source: C, destination: Pro
#edges: 2839201
relation3: source: C, destination: Pro
#edges: 2904592
relation4: source: C, destination: Pro
#edges: 1634362
relation5: source: C, destination: G
#edges: 1311251
relation6: source: C, destination: G
#edges: 5084698
relation7: source: C, destination: G
#edges: 5391665
relation8: source: C, destination: G
#edges: 2767502
relation9: source: C, destination: G
#edges: 733229
Total number of edges: 25429957
Writing graph file "pbg_graph.txt" to "./gnn_result_100/pbg/graph0" ...
Finished.
Auto-estimated weight decay is 0.001393
`.settings.pbg_params['wd']` has been updated to 0.001393
Converting input data ...
[2025-12-17 18:22:53.988113] Using the 10 relation types given in the config
[2025-12-17 18:22:53.988972] Searching for the entities in the edge files...
[2025-12-17 18:23:25.030934] Entit

/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/monae_simba/lib/python3.10/site-packages/torchbiggraph/tensorlist.py:86: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  if data.numel() == 0 and data.storage().size() == 0:


[2025-12-17 18:26:05.548020] - Processed 25429957 edges in total
Starting training ...


/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/monae_simba/lib/python3.10/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/monae_simba/lib/python3.10/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/monae_simba/lib/python3.10/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If 

Finished
Time taken: 3012.643848657608
Performing softmax transformation for query data 0;
Performing softmax transformation for query data 1;
#shared features: 50
Performing randomized SVD ...
(2000, 50)
(134, 50)
Searching for mutual nearest neighbors ...
6700 edges are selected


/ictstr01/groups/ml01/code/anastasia.litinetskaya/multigrate_reproducibility/imputation/monae/../../../monae/src/emb_init/tools/_integration.py:135: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata_ref_query = ad.AnnData(X=sim_ref_query,


MultiDiGraph with 2134 nodes and 15534 edges
[INFO] check_graph: Checking variable coverage...
[INFO] check_graph: Checking edge attributes...
[INFO] check_graph: Checking self-loops...
[INFO] check_graph: Checking graph symmetry...
[INFO] check_graph: All checks passed!
